In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\pc world\Telco-Customer-Churn.csv")

In [4]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [5]:
df[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']] = df[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']].replace('No internet service', 'No')

In [6]:
cont = ['SeniorCitizen','tenure',"MonthlyCharges","TotalCharges"]
ordi = ["Contract"]
nomi = ['Churn','gender','Partner','Dependents','PhoneService',"MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","PaperlessBilling","PaymentMethod",]

In [7]:
from sklearn.impute import SimpleImputer

In [8]:
cont_impute = SimpleImputer(strategy='mean')
ordi_impute = SimpleImputer(strategy='most_frequent')
nomi_impute = SimpleImputer(strategy='most_frequent')

In [9]:
df[cont] = cont_impute.fit_transform(df[cont])
df[ordi] = ordi_impute.fit_transform(df[ordi])
df[nomi] = nomi_impute.fit_transform(df[nomi])

In [10]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder

In [11]:
ordinal  = {
    "Contract":['Month-to-month', 'One year', 'Two year'],
      }

In [12]:
oe = OrdinalEncoder(categories=[ordinal["Contract"]],                                
                   dtype='int')

In [13]:
df[ordi] = oe.fit_transform(df[ordi])

In [14]:
ohe = OneHotEncoder(drop = 'first',sparse_output=True)

In [15]:
df1 = pd.DataFrame(ohe.fit_transform(df[nomi]).toarray(),columns=ohe.get_feature_names_out())

In [16]:
df.drop(nomi,axis =1 ,inplace = True)

In [17]:
df = pd.concat([df,df1],axis = 1)

In [18]:
df.drop("customerID",axis = 1, inplace = True)

In [19]:
df.head()

,SeniorCitizen,tenure,Contract,MonthlyCharges,TotalCharges,Churn_Yes,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0.0,1.0,0,29.85,29.85,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,34.0,1,56.95,1889.50,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,2.0,0,53.85,108.15,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,45.0,1,42.30,1840.75,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,0,70.70,151.65,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [20]:
X = df.drop("Churn_Yes",axis = 1)
y = df['Churn_Yes']

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import roc_auc_score,make_scorer

In [22]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)

#### LR

In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test,y_pred)

0.8076650106458482

In [24]:
y_prob = lr.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_prob)

0.8400901605169898

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   float64
 1   tenure                                 7043 non-null   float64
 2   Contract                               7043 non-null   int32  
 3   MonthlyCharges                         7043 non-null   float64
 4   TotalCharges                           7043 non-null   float64
 5   Churn_Yes                              7043 non-null   float64
 6   gender_Male                            7043 non-null   float64
 7   Partner_Yes                            7043 non-null   float64
 8   Dependents_Yes                         7043 non-null   float64
 9   PhoneService_Yes                       7043 non-null   float64
 10  MultipleLines_No phone service         7043 non-null   float64
 11  Mult

#### ML Model:

In [44]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

data = pd.DataFrame(columns=range(23))  

numerical_features = list(range(data.shape[1]))

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features)
    ]
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [45]:
X = df.drop('Churn_Yes',axis = 1)
y = df['Churn_Yes']

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [47]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=23)

In [48]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
accuracy_score(y_test,y_pred)


0.8026969481902059

In [49]:
import pickle

In [50]:
pickle.dump(model,open("prediction_pl.pkl","wb"))

In [51]:
model_pickled = pickle.load(open("prediction_pl.pkl","rb"))

In [54]:
model.feature_names_in_

array(['SeniorCitizen', 'tenure', 'Contract', 'MonthlyCharges',
       'TotalCharges', 'gender_Male', 'Partner_Yes', 'Dependents_Yes',
       'PhoneService_Yes', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_Yes', 'OnlineBackup_Yes',
       'DeviceProtection_Yes', 'TechSupport_Yes', 'StreamingTV_Yes',
       'StreamingMovies_Yes', 'PaperlessBilling_Yes',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype=object)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   float64
 1   tenure                                 7043 non-null   float64
 2   Contract                               7043 non-null   int32  
 3   MonthlyCharges                         7043 non-null   float64
 4   TotalCharges                           7043 non-null   float64
 5   Churn_Yes                              7043 non-null   float64
 6   gender_Male                            7043 non-null   float64
 7   Partner_Yes                            7043 non-null   float64
 8   Dependents_Yes                         7043 non-null   float64
 9   PhoneService_Yes                       7043 non-null   float64
 10  MultipleLines_No phone service         7043 non-null   float64
 11  Mult